In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train_sample = pd.read_csv(path + 'train_sample_cleaned.csv')

In [3]:
train_sample.dtypes

ip               int64
app              int64
device           int64
os               int64
channel          int64
year             int64
month            int64
week             int64
day              int64
hour             int64
timestamp        int64
minute           int64
second           int64
is_attributed    int64
dtype: object

# explore time

In [4]:
print('training')
print(train_sample.timestamp.max() - train_sample.timestamp.min())
print('max: {}'.format(train_sample.timestamp.max()))
print('min: {}'.format(train_sample.timestamp.min()))
print('==========')
print('testing')
print(train_sample.timestamp.max() - train_sample.timestamp.min())
print('max: {}'.format(train_sample.timestamp.max()))
print('min: {}'.format(train_sample.timestamp.min()))

training
259191
max: 1510243191
min: 1509984000
testing
259191
max: 1510243191
min: 1509984000


# Use 10% train as Sample

In [5]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

ratio = 0.05
length = int(len(train) * ratio)
train = train[-length:None]

# del train
gc.collect()

0

# Get K Fold

In [6]:
from sklearn.model_selection import KFold
K = 6
kf = KFold(n_splits=K, shuffle = False)

history_index = []
train_index = []
for h,t in kf.split(train):
    history_index.append(h)
    train_index.append(t)

In [7]:
df_history = train.iloc[history_index[-1]].copy()
df_train = train.iloc[train_index[-1]].copy()

import sys
print(sys.getsizeof(df_train)/ 1024 **3)
print(len(df_train))

0.14924442768096924
1540865


# Get Group

In [7]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'timestamp',
              'minute',
              'second']
# for col in feature_col:
#     orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
    
for col in feature_col:
    orders[col] = 10 ** (int(np.log(train_sample[col].max() + 1) / np.log(10)) + 1)    
    
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the combination will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

import gc
# del train
gc.collect()

60

In [5]:
orders

{'app': 1000,
 'channel': 1000,
 'day': 100,
 'device': 10000,
 'hour': 100,
 'ip': 1000000,
 'minute': 100,
 'os': 1000,
 'second': 100,
 'timestamp': 10000000000}

# Combine Features
below is just an example

In [6]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0
for cols in combinations(combine_col, 2):
    counter += 1
print(counter)

36


In [7]:
combines = []
for comb in combinations(combine_col, 2):
    combines.append(comb)

## count
plan 1. count from historical data  
plan 2. count from all data

In [47]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

cols = combines[1]
feature_name = col_name(cols, func=count)
print(feature_name)
df_train[feature_name] = count(df_history, df_train, cols)

ip_device_count


# count and sort

In [24]:
def countsort(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts().iloc[::-1]
    count_map.iloc[:] = list(range(1, len(count_map) + 1))
    
    return group.map(count_map).fillna(-1)

cols = combines[18]
feature_name = col_name(cols, func=countsort)
print(feature_name)
train_sample[feature_name] = countsort(train_sample, train_sample, cols)

device_hour_countsort


# running count
calculate the order of the occurence  
meaning: 1st time occur, mark 1. next then add 1

In [50]:
from collections import defaultdict 


    
def runningcount(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    result = []
    index_train = df_train.index.values
    group_all = get_group(df_history, cols)
    count_map = group_all.value_counts()

    urls_d = defaultdict(int)
    for url in group:
        urls_d[url] += 1
        result.append(urls_d[url])
    result = np.array(result)
    return pd.Series(result) / group_all.map(count_map)
    
cols = combines[18]
feature_name = col_name(cols, func=runningcount)
print(feature_name)
train_sample[feature_name] = runningcount(train_sample, train_sample, cols)

device_hour_runningcount


In [51]:
train_sample

,ip,app,device,os,channel,year,month,week,day,hour,timestamp,minute,second,is_attributed,device_hour_countsort,device_hour_runningcount
0,87540,12,1,13,497,2017,11,45,7,9,1510047038,30,38,0,278,0.000219
1,105560,25,1,17,259,2017,11,45,7,13,1510062027,40,27,0,288,0.000194
2,101424,12,1,19,212,2017,11,45,7,18,1510077924,5,24,0,272,0.000894
3,94584,13,1,13,477,2017,11,45,7,4,1510030688,58,8,0,292,0.000173
4,68413,12,1,1,178,2017,11,45,9,9,1510218009,0,9,0,278,0.000439
5,93663,3,1,17,115,2017,11,45,9,1,1510190533,22,13,0,284,0.000204
6,17059,1,1,17,135,2017,11,45,9,1,1510190278,17,58,0,284,0.000408
7,121505,9,1,25,442,2017,11,45,7,10,1510048913,1,53,0,287,0.000195
8,192967,2,2,22,364,2017,11,45,8,9,1510133717,35,17,0,262,0.004292
9,143636,3,1,19,135,2017,11,45,8,12,1510144526,35,26,0,280,0.000206


In [23]:
train_sample

,ip,app,device,os,channel,year,month,week,day,hour,timestamp,minute,second,is_attributed,device_hour_countsort
0,87540,12,1,13,497,2017,11,45,7,9,1510047038,30,38,0,278
1,105560,25,1,17,259,2017,11,45,7,13,1510062027,40,27,0,288
2,101424,12,1,19,212,2017,11,45,7,18,1510077924,5,24,0,272
3,94584,13,1,13,477,2017,11,45,7,4,1510030688,58,8,0,292
4,68413,12,1,1,178,2017,11,45,9,9,1510218009,0,9,0,278
5,93663,3,1,17,115,2017,11,45,9,1,1510190533,22,13,0,284
6,17059,1,1,17,135,2017,11,45,9,1,1510190278,17,58,0,284
7,121505,9,1,25,442,2017,11,45,7,10,1510048913,1,53,0,287
8,192967,2,2,22,364,2017,11,45,8,9,1510133717,35,17,0,262
9,143636,3,1,19,135,2017,11,45,8,12,1510144526,35,26,0,280


In [18]:
count_map.iloc[::-1].index.values

array([   605,  88309,  10010,    607,  58115, 333117,   5917, 303220,
        12902,  32111, 108009,   7907,   9702,  57912,    612,  10913,
         4903,  54911,  18222,   4007,  10914,  55802, 131815,  12409,
         5918,  11408,  11600,  18013,   5911,   3607,   1623,  18207,
         2007,    600,  10200,   1816,    409,    601, 172813,   1113,
        55809,    411,  19602,   5913,   2011,   5914,   5610,  10604,
          617,   9714, 354522,  21117,  55213,   2105,  26805,  29110,
        60709,  24115,   7606,  18800,  36202,  48620,  12410,   2111,
       148214,   3007,   5308, 104204,   6012,  16318,  21119,   5814,
        38502,   7402, 242908,    619,   3306,   9711, 183922, 386711,
         4010, 328205,   3308,  92803,   4015,    623,   5613,   4014,
        37411,   7410,   1713,  15413,  22008,   3314,  82813,   5910,
        20214,   2506,   5011,    913,   2512,  41413,  42006,    401,
         5007,   1602,  59610,   7808,  34710,  38609, 298005,   1606,
      

In [20]:
count_map

605         1
88309       2
10010       3
607         4
58115       5
333117      6
5917        7
303220      8
12902       9
32111      10
108009     11
7907       12
9702       13
57912      14
612        15
10913      16
4903       17
54911      18
18222      19
4007       20
10914      21
55802      22
131815     23
12409      24
5918       25
11408      26
11600      27
18013      28
5911       29
3607       30
         ... 
215       263
210       264
214       265
213       266
211       267
212       268
120       269
121       270
119       271
118       272
122       273
117       274
116       275
123       276
108       277
109       278
115       279
112       280
106       281
111       282
107       283
101       284
102       285
103       286
110       287
113       288
105       289
114       290
100       291
104       292
Length: 292, dtype: int64

# mean
mean P(target | feature combination)

# purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer))
Get the conditional Probability only from historical data and apply to train data.

P(replay | X feature combination) = P( replay & X feature combination) / P (X feature combination)
=(count(replay & X feature combination) / count(total)) / (count(X feature combination) / count(total))
= count(replay & X feature combination) / count(X feature combination)
= sum((replay & X feature combination)) / count(X feature combination)
= sum((replay or not replayed & X feature combination)) / count(X feature combination)# since replay is 1, not replay is 0
= sum( X feature combination) / count(X feature combination)
= mean(X feature combination)

In [52]:
def scaller(num):
    sca = 1
    while num * sca < 1:
        sca *= 10
    return sca

def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    mean_map = df_history.groupby(group_history)[target].mean()
    # mean_map: key - encoding, value - target mean
    ### sca
    m_min = mean_map[mean_map > 0].min()
    sca = scaller(m_min)
    mean_map *= sca
    ###
    return group.map(mean_map).fillna(-1)

cols = combines[1]
feature_name = col_name(cols, func=mean)
print(feature_name)
df_train[feature_name] = mean(df_history, df_train, cols, target='is_attributed')

ip_device_mean


KeyboardInterrupt: 

# reversemean
reverse mean P(feature combination | target)

In [13]:
def reversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)

cols = combines[1]
feature_name = col_name(cols, func=reversemean)
print(feature_name)
df_train[feature_name] = reversemean(df_history, df_train, cols, target='is_attributed')

ip_device_reversemean


# time2nextclick

In [16]:
def time2nextclick(df_history, df_train, cols, target, timecol='timestamp'):
    
    result = []
    df_reverse = df_train.sort_index(ascending=False)
    df_reverse = df_train.sort_values([timecol], ascending=False)
    group = get_group(df_reverse,  cols)
    
    next_heard = {}
    for g, t in zip(group, df_reverse[timecol]):
        if g in next_heard:
            result.append(next_heard[g] - t)
        else:
            result.append(-1)
        next_heard[g] = t
    
    result.reverse()
    return result

cols = combines[1]
feature_name = col_name(cols, func=time2nextclick)
print(feature_name)
df_train[feature_name] = time2nextclick(df_history, df_train, cols, target='is_attributed')

ip_device_time2nextclick


# time from previous click

In [20]:
def time2previousclick(df_history, df_train, cols, target, timecol='timestamp'):
    
    result = []
    group = get_group(df_train, cols)

    last_heard = {}
    for t, g in zip(df_train[timecol], group):
        if g in last_heard:
            result.append(t - last_heard[g])
        else:
            result.append(-1)
        last_heard[g] = t
        
    return result

cols = combines[1]
feature_name = col_name(cols, func=time2previousclick)
print(feature_name)
df_train[feature_name] = time2previousclick(df_history, df_train, cols, target='is_attributed')

ip_device_time2previousclick


# count from past

In [22]:
def countfrompast(df_history, df_train, cols, target, timecol='timestamp'):
    
    group = get_group(df_train, cols)
    
    count = {}
    result = []
    for g in group.values:
        if g not in count:
            count[g] = 0
        else:
            count[g] += 1
        result.append(count[g])
        
    return result

cols = combines[1]
feature_name = col_name(cols, func=countfrompast)
print(feature_name)
df_train[feature_name] = countfrompast(df_history, df_train, cols, target='is_attributed')

ip_device_countfrompast


# count from future

In [24]:
def countfromfuture(df_history, df_train, cols, target, timecol='timestamp'):
    
    result = []
    df_reverse = df_train.sort_values([timecol], ascending=False)
    group = get_group(df_reverse,  cols)
    
    count = {}
    for g in group.values:
        if g in count:
            result.append(count[g])
            count[g] += 1 
        else:
            result.append(0)
            count[g] = 1
    
    result.reverse()
    return result

cols = combines[1]
feature_name = col_name(cols, func=countfromfuture)
print(feature_name)
df_train[feature_name] = countfromfuture(df_history, df_train, cols, target='is_attributed')

ip_device_countfromfuture


# last time diff

In [27]:
def lasttimediff(df_history, df_train, cols, target, timecol='timestamp'):
    
    group = get_group(df_train, cols)
        
    last_time = df_train.groupby(group)[timecol].last()
    
    return group.map(last_time) - df_train[timecol]

def lasttimediff(df_history, df_train, cols, target, timecol='timestamp'):
    
    group = get_group(df_train, cols)
        
    last_time = df_train.groupby(group)[timecol].last()
    
    return group.map(last_time) - df_train[timecol]

cols = combines[1]
feature_name = col_name(cols, func=lasttimediff)
print(feature_name)
df_train[feature_name] = lasttimediff(df_history, df_train, cols, target='is_attributed')

ip_device_lasttimediff


In [28]:
df_train

,ip,app,device,os,channel,day,hour,timestamp,minute,second,is_attributed,intesthh,ip_device_time2nextclick,ip_device_time2previousclick,ip_device_countfrompast,ip_device_countfromfuture,ip_device_lasttimediff
183363025,71464,12,1,19,328,9,15,1510241240,27,20,0,2,143,-1,0,44,1831
183363026,111025,13,1,13,477,9,15,1510241240,27,20,0,2,8,-1,0,15,1959
183363027,97514,3,1,18,211,9,15,1510241240,27,20,0,2,3,-1,0,19,1896
183363028,119558,9,1,19,445,9,15,1510241240,27,20,0,2,0,-1,0,49,1944
183363029,61863,18,1,13,121,9,15,1510241240,27,20,0,2,23,-1,0,122,73
183363030,71808,1,1,12,101,9,15,1510241240,27,20,0,2,0,-1,0,1269,1958
183363031,63246,18,1,13,439,9,15,1510241240,27,20,0,2,1,-1,0,37,1948
183363032,4683,21,1,19,128,9,15,1510241240,27,20,0,2,99,-1,0,15,1941
183363033,12971,3,1,19,137,9,15,1510241240,27,20,0,2,9,-1,0,1008,1921
183363034,42283,12,1,18,259,9,15,1510241240,27,20,0,2,0,-1,0,61,1339


# generate all cols

In [140]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0
for num_col in [1,2,3]:
    for cols in combinations(combine_col, num_col):
        for func in [count, mean]:
            counter += 1
            feature_name = col_name(cols, func=func)
            df_train[feature_name] = func(df_history, df_train, cols, target='is_attributed')
            print('{}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3))

1:   ip_count   			 size: 0.36737098544836044 G.
2:   ip_mean   			 size: 0.3903316706418991 G.
3:   app_count   			 size: 0.4132923558354378 G.
4:   app_mean   			 size: 0.43625304102897644 G.
5:   device_count   			 size: 0.4592137262225151 G.
6:   device_mean   			 size: 0.48217441141605377 G.
7:   os_count   			 size: 0.5051350966095924 G.
8:   os_mean   			 size: 0.5280957818031311 G.
9:   channel_count   			 size: 0.5510564669966698 G.
10:   channel_mean   			 size: 0.5740171521902084 G.
11:   day_count   			 size: 0.5969778373837471 G.
12:   day_mean   			 size: 0.6199385225772858 G.
13:   hour_count   			 size: 0.6428992077708244 G.
14:   hour_mean   			 size: 0.6658598929643631 G.
15:   minute_count   			 size: 0.6888205781579018 G.
16:   minute_mean   			 size: 0.7117812633514404 G.
17:   second_count   			 size: 0.7347419485449791 G.
18:   second_mean   			 size: 0.7577026337385178 G.
19:   ip_app_count   			 size: 0.7806633189320564 G.
20:   ip_app_mean   			 size: 0.803624

146:   ip_minute_second_mean   			 size: 3.696670338511467 G.
147:   app_device_os_count   			 size: 3.7196310237050056 G.
148:   app_device_os_mean   			 size: 3.7425917088985443 G.
149:   app_device_channel_count   			 size: 3.765552394092083 G.
150:   app_device_channel_mean   			 size: 3.7885130792856216 G.
151:   app_device_day_count   			 size: 3.8114737644791603 G.
152:   app_device_day_mean   			 size: 3.834434449672699 G.
153:   app_device_hour_count   			 size: 3.8573951348662376 G.
154:   app_device_hour_mean   			 size: 3.8803558200597763 G.
155:   app_device_minute_count   			 size: 3.903316505253315 G.
156:   app_device_minute_mean   			 size: 3.9262771904468536 G.
157:   app_device_second_count   			 size: 3.9492378756403923 G.
158:   app_device_second_mean   			 size: 3.972198560833931 G.
159:   app_os_channel_count   			 size: 3.9951592460274696 G.
160:   app_os_channel_mean   			 size: 4.018119931221008 G.
161:   app_os_day_count   			 size: 4.041080616414547 G.
162: 

# multi-processing version

In [25]:
import multiprocessing

workers = 3

def _process_each_col(kwargs_list):
    print('enter here')
    result_dict = {}
    for kwargs in kwargs_list:
        df_train = kwargs.get('df_train')
        df_history = kwargs.get('df_history')
        target = kwargs.get('target')
        cols = kwargs.get('cols')
        func = kwargs.get('func')
        feature_name = col_name(cols, func=func)
        print(feature_name)
        result_dict[feature_name] = func(df_history, df_train, cols, target=target)
    return result_dict

#### get feature cols:
feature_cols = []
for num_col in [1,2,3]:
    for cols in combinations(combine_col, num_col):
        feature_cols.append(cols)
print('get feature cols done!')
##########################################

#### build kwargs:
combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

func_pool = [count, mean]

def build_kwargs(df_history, df_train, target, combine_col, func_pool, comb_total=3):
    kwargs_pool = []
    for num_col in range(1, comb_total + 1):
        for cols in combinations(combine_col, num_col):
            for func in func_pool:
                kwargs = {}
                kwargs['df_history'] = df_history
                kwargs['df_train'] = df_train
                kwargs['target'] = target
                kwargs['cols'] = cols
                kwargs['func'] = func
                kwargs_pool.append(kwargs)
    return kwargs_pool
target = 'is_attributed'
kwargs_pool = build_kwargs(df_history, df_train, target, combine_col, func_pool, comb_total=3)
print('build kwargs done!')
################################################################################################

pool = multiprocessing.Pool(processes=workers)
result = pool.map(_process_each_col, [kwargs for kwargs in np.array_split(kwargs_pool, workers)])
pool.close()

for each_thread in result:
    for key in each_thread:
        df_train[key] = each_thread[key]


get feature cols done!
build kwargs done!
enter here
os_mean
ip_count
ip_mean
app_count
app_mean
device_count
device_mean
os_count
channel_count
channel_mean
day_count
day_mean
hour_count
hour_mean
minute_count
minute_mean
second_count
second_mean
ip_app_count
ip_app_mean
ip_app_device_mean
ip_device_count
ip_device_mean
ip_os_count
enter here
hour_second_count
ip_os_mean
hour_second_mean
minute_second_count
minute_second_mean
ip_channel_count
ip_app_device_count
ip_channel_mean
ip_app_os_count
ip_day_count
ip_day_mean
ip_hour_count
ip_app_os_mean
ip_hour_mean
ip_minute_count
ip_minute_mean
ip_app_channel_count
ip_second_count
ip_app_channel_mean
enter here
app_channel_minute_count
app_channel_minute_mean
ip_second_mean
app_channel_second_count
app_channel_second_mean
app_day_hour_count
ip_app_day_count
app_day_hour_mean
app_device_count
app_day_minute_count
app_device_mean
app_day_minute_mean
ip_app_day_mean
app_os_count
app_day_second_count
app_os_mean
app_day_second_mean
app_channel

MaybeEncodingError: Error sending result: '[{'ip_count': 183363025      252.0
183363026    21724.0
183363027      271.0
183363028      540.0
183363029       80.0
183363030     6855.0
183363031       45.0
183363032      312.0
183363033      816.0
183363034       95.0
183363035     2198.0
183363036      128.0
183363037      323.0
183363038      363.0
183363039    21724.0
183363040      942.0
183363041      699.0
183363042     6855.0
183363043    12042.0
183363044      101.0
183363045     1911.0
183363046      236.0
183363047       68.0
183363048     1224.0
183363049      926.0
183363050      104.0
183363051      155.0
183363052       58.0
183363053      548.0
183363054     2081.0
              ...   
184903860     1518.0
184903861      166.0
184903862       96.0
184903863      194.0
184903864     2501.0
184903865      213.0
184903866       39.0
184903867     4437.0
184903868       38.0
184903869      275.0
184903870      773.0
184903871      820.0
184903872       48.0
184903873       51.

In [26]:
splitter = np.array_split(df_train, 30)

In [27]:
splitter

[               ip  app  device  os  channel  day  hour   timestamp  minute  \
 183363025   71464   12       1  19      328    9    15  1510241240      27   
 183363026  111025   13       1  13      477    9    15  1510241240      27   
 183363027   97514    3       1  18      211    9    15  1510241240      27   
 183363028  119558    9       1  19      445    9    15  1510241240      27   
 183363029   61863   18       1  13      121    9    15  1510241240      27   
 183363030   71808    1       1  12      101    9    15  1510241240      27   
 183363031   63246   18       1  13      439    9    15  1510241240      27   
 183363032    4683   21       1  19      128    9    15  1510241240      27   
 183363033   12971    3       1  19      137    9    15  1510241240      27   
 183363034   42283   12       1  18      259    9    15  1510241240      27   
 183363035   80660   15       1  22        3    9    15  1510241240      27   
 183363036   51558   13       1  19      477    9   

In [27]:
len(splitter[0])

9

In [49]:
df_train

,ip,app,device,os,channel,day,hour,timestamp,minute,second,is_attributed,ip_device_count,ip_device_mean
183363025,71464,12,1,19,328,9,15,1510241240,27,20,0,249.0,0.000000
183363026,111025,13,1,13,477,9,15,1510241240,27,20,0,19183.0,0.000000
183363027,97514,3,1,18,211,9,15,1510241240,27,20,0,268.0,0.000000
183363028,119558,9,1,19,445,9,15,1510241240,27,20,0,477.0,0.000000
183363029,61863,18,1,13,121,9,15,1510241240,27,20,0,78.0,0.000000
183363030,71808,1,1,12,101,9,15,1510241240,27,20,0,6021.0,0.000166
183363031,63246,18,1,13,439,9,15,1510241240,27,20,0,45.0,0.000000
183363032,4683,21,1,19,128,9,15,1510241240,27,20,0,302.0,0.000000
183363033,12971,3,1,19,137,9,15,1510241240,27,20,0,693.0,0.000000
183363034,42283,12,1,18,259,9,15,1510241240,27,20,0,93.0,0.000000


In [53]:
def scaller(num):
    sca = 1
    while num * sca < 1:
        sca *= 10
    return sca

target='is_attributed'
group = get_group(df_train, cols)
# encoding df_history's cols into a new series
group_history = get_group(df_history, cols)
# get the conditional probability p(target| feature combination. eg, artist_name_composer) 
mean_map = df_history.groupby(group_history)[target].mean()
# mean_map: key - encoding, value - target mean
### sca
m_min = mean_map[mean_map > 0].min()
sca = scaller(m_min)
mean_map *= sca

In [56]:
mean_map[mean_map > 0].sort_values()

482820001          4.802382
1142760001         4.910868
1193490001         5.806189
482120001          9.431293
798570001          9.858044
534540001          9.911785
482400001          9.960159
935870001         10.895620
798270001         10.909884
539640001         11.479738
735160002         13.648151
481700001         14.675668
882810001         15.622559
867670001         16.131634
718080001         16.608537
1088810001        16.708438
957660001         17.489652
537150001         18.317255
438270001         18.392496
1193690001        19.243105
799090001         19.275251
1252220001        20.364525
39640001          23.369946
382650001         24.224806
383000001         24.473813
57290001          24.734108
382190001         25.846472
591250001         26.343519
1239940001        27.338838
1183390001        27.419797
                  ...      
2952780000    100000.000000
2952960018    100000.000000
3311770000    100000.000000
2953110000    100000.000000
2952520001    100000

In [43]:
df = pd.DataFrame({'col1': [0,1,2,3,4,5,6]}, )
df = df.rename({0:2,1:4,2:6,3:8,4:10,5:12,6:2})

In [44]:
df

,col1
2,0
4,1
6,2
8,3
10,4
12,5
2,6


In [49]:
df.iloc[3]

col1    3
Name: 8, dtype: int64

# matrix factorization

In [71]:
from lightfm import LightFM
import scipy.sparse as sp
from scipy.sparse import coo_matrix
from lightfm import LightFM
from sklearn.preprocessing import LabelEncoder

def matrix_factorization(df, df_history, target, item_col, userid_col, userraw_col):
    """
    userid_col is unique user id
    item_col is unique itme id
    userraw_col is used to construct user feature. dim: user_id*userraw
    """
    dff = pd.DataFrame()
    dff_history = pd.DataFrame()


    #1. process item
    if item_col is None:
        dff['item'] = np.zeros(len(df))
        dff_history['item'] = np.zeros(len(df_history))
    else:
        encoder = LabelEncoder()
        group = get_group(df, item_col)
        group_history = get_group(df_history, item_col)
        encoder.fit(pd.concat([group, group_history]))
        dff['item'] = encoder.transform(group)
        dff_history['item'] = encoder.transform(group_history)
#     print('processing item done!')

    #2. user raw
    group = get_group(df, userraw_col)
    group_history = get_group(df_history, userraw_col)
    encoder = LabelEncoder()
    encoder.fit(pd.concat([group, group_history]))
    dff['userraw'] = encoder.transform(group)
    dff_history['userraw'] = encoder.transform(group_history)
#     print('processing user raw done')


    #3. user_id
    group = get_group(df, userid_col)
    group_history = get_group(df_history, userid_col)
    encoder = LabelEncoder()
    encoder.fit(pd.concat([group, group_history]))
    dff['user_id'] = encoder.transform(group)
    dff_history['user_id'] = encoder.transform(group_history)
#     print('processing user id done')



    num_users = max(dff.user_id.max(), dff_history.user_id.max()) + 1
    num_items = max(dff.item.max(), dff_history.item.max()) + 1
    num_userraw = max(dff.userraw.max(), dff_history.userraw.max()) + 1

    M = coo_matrix(
            (df_history[target], ( dff_history.user_id, dff_history.item)),
            shape=(num_users, num_items)
        )

    user_features = pd.concat([dff, dff_history])[['userraw', 'user_id']].drop_duplicates()

    user_features = coo_matrix(
        (np.ones(len(user_features)), (user_features.user_id, user_features.userraw)),
        shape=(num_users, num_userraw)
    )

    user_features = sp.hstack([sp.eye(num_users), user_features])

    model = LightFM(no_components=50, learning_rate=0.1)

    model.fit(
            M, 
            epochs=2, 
            num_threads=50, 
            user_features=user_features,
        )
    result = model.predict(
        dff.user_id.values, 
        dff.item.values, 
        user_features=user_features,
    )
    return result

target = 'is_attributed'
item_col = ['app','device', 'os']
# item_col = None
userid_col = ['ip', 'os', 'device','app', 'channel']
userraw_col = ['ip']

result = matrix_factorization(train_sample, train_sample,target, item_col, userid_col, userraw_col)

In [72]:
result

array([-4.94598484, -2.42577314, -4.8414197 , ..., -5.04750443,
       -3.2668891 , -2.95993662])

In [68]:
view = pd.DataFrame()
view['preds'] = result
view['ori'] = train_sample.is_attributed.values

In [69]:
view[view.ori == 1]

,preds,ori
284,-0.346638,1
481,0.222761,1
1208,-0.763565,1
1341,-0.779415,1
1412,-0.785339,1
1666,0.256924,1
1771,0.348931,1
1917,0.232380,1
3914,0.274352,1
3992,0.686690,1


In [22]:
num_userraw

34857